In [60]:
import csv
import numpy as np 
import nltk
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from time import time 
from sklearn.model_selection import train_test_split

from tensorflow import keras 
from tensorflow.keras import layers
#from tensorflow.keras import Sequential
#from tensorflow.keras.layers import Dense
#from tensorflow.keras.layers import LSTM, Dropout, Embedding
#from tensorflow.keras.layers import Bidirectional 
from sklearn.metrics import confusion_matrix, f1_score 

In [61]:
# 2번 블럭
# 파라미터 설정
MY_VOCAB = 5000   # 내가 사용할 단어 수, 인기도 기준
MY_EMBED = 64     # 임베딩 차원
MY_HIDDEN = 100   # RNN 셀의 규모
MY_LEN = 200      # 기사의 길이

MY_SPLIT = 0.8    # 학습용 데이터의 비율
MY_SAMPLE = 121   # 샘플 기사
MY_EPOCH = 10    # 반복 학습 수


In [62]:
#제외어 (Stopword) 설정 

nltk.download('stopwords')
MY_STOP = set(nltk.corpus.stopwords.words('english'))

print('제외어:')
print(MY_STOP)
print('제외어 갯수:', len(MY_STOP))


제외어:
{'the', 'hers', 'ourselves', "shan't", 'when', 'should', 'is', "it's", 'to', 'their', 'with', "you'd", 'them', 'there', 'who', 'm', 'if', 'having', 'after', 'have', 'below', 'mustn', "won't", 'very', 'before', 'these', 'further', "mightn't", 'themselves', 'whom', 't', 'between', 'itself', 'again', "you're", 'nor', 'only', 'your', 'o', 'or', "doesn't", 'not', 'some', 'any', 'myself', 'a', 'all', 'such', "you'll", 'too', "isn't", 'doing', 'ain', 'will', "didn't", 'yourself', 'didn', 'what', 'same', 'it', "that'll", 'does', 'herself', 'she', 'did', 're', 'theirs', 'am', "haven't", 'been', 'on', 'which', 'an', 'during', 'both', 'into', 'once', 'had', 'here', 'than', "weren't", "needn't", 'above', 'they', 'mightn', "wasn't", 'of', 'll', 'was', 'haven', 'doesn', "mustn't", 'over', 'her', 'where', 'most', 'at', 'under', 've', 'for', "aren't", 'has', 'this', 'won', 'out', 'other', 'hasn', 'why', "hadn't", 'how', 'so', 'were', 'then', 'now', 'from', 'while', 'few', 'don', 'himself', 'those

In [63]:
# 4번 블럭
# 데이터 보관 창고
original = []
articles = []
labels = []

print(type(original))

<class 'list'>


In [64]:
with open('bbc-text.csv', 'r') as file:
    

    # 컬럼 이름 읽기
    reader = csv.reader(file)
    header = next(reader)
    print(header)

    # 기사 한줄씩 처리
    for row in reader:
        labels.append(row[0])
        original.append(row[1])

        # 제외어 삭제하기
        news = row[1]
        #print('전:', news)
        for word in MY_STOP:
            mask = ' ' + word + ' '
            news = news.replace(mask, ' ')
        #print('후:', news)

        articles.append(news)
print('처리한 기사 숫자:', len(articles))

['category', 'text']
처리한 기사 숫자: 2225


In [65]:
#샘플기사 원본 출력

print('샘플 기사 원본')
print(original[MY_SAMPLE])
print(labels[MY_SAMPLE])
print(type(original[MY_SAMPLE]))
print('총 단어 수:', len(original[MY_SAMPLE].split()))

샘플 기사 원본
jack cunningham to stand down veteran labour mp and former cabinet minister jack cunningham has said he will stand down at the next election.  one of the few blair-era ministers to serve under jim callaghan  he was given the agriculture portfolio when labour regained power in 1997. mr cunningham went on to become tony blair s  cabinet enforcer . he has represented the constituency now known as copeland since 1970. mr blair said he was a  huge figure  in labour and a  valued  personal friend .  during labour s long period in opposition  mr cunningham held a number of shadow roles including foreign affairs  the environment and as trade spokesman. as agriculture minister he caused controversy when he decided to ban beef on the bone in the wake of fears over bse. he quit the government in 1999 and in recent years has served as the chairman of the all-party committee on lords reform and has been a loyal supporter of the government from the backbenches.
politics
<class 'str'>
총 단어 수

In [66]:
#7번 블럭
#제외어 삭제 기사 출력
print('제외어 삭제본')
print(articles[MY_SAMPLE])
print('총 단어 수:', len(articles[MY_SAMPLE].split()))
print(type(articles[MY_SAMPLE]))



제외어 삭제본
jack cunningham stand veteran labour mp former cabinet minister jack cunningham said stand next election.  one blair-era ministers serve jim callaghan  given agriculture portfolio labour regained power 1997. mr cunningham went become tony blair  cabinet enforcer . represented constituency known copeland since 1970. mr blair said  huge figure  labour  valued  personal friend .  labour long period opposition  mr cunningham held number shadow roles including foreign affairs  environment trade spokesman. agriculture minister caused controversy decided ban beef bone wake fears bse. quit government 1999 recent years served chairman all-party committee lords reform loyal supporter government backbenches.
총 단어 수: 95
<class 'str'>


In [67]:
for i in range(len(articles)):
    if ' the ' in articles[i]:
        print('원본: ', original[i])
        print('처리: ', articles[i])

원본:  goldsmith:  i was not leant on  the attorney general has again denied being  leant on  by downing street to make the legal case for invading iraq.  claims a written answer on the legality of the war was drafted by downing street were  wholly unfounded   he insisted during stormy lords exchanges. lord goldsmith said the answer represented his  genuinely held independent view  the war was legal. the text was released on the eve of a crucial commons vote in which mps backed the invasion of iraq. many labour mps have since indicated that the attorney general s answer played a pivotal role in their willingness to back the conflict. the government has resisted calls to publish the full advice  saying such papers are always kept confidential. in the house of lords  the attorney general faced a call by former tory lord chancellor lord mackay to now publish the  full text  of the advice - the suggestion was rejected. another peer meanwhile  lord skidelsky  said not to publish the full lega

In [68]:
#기사 Tokenization 처리 

A_token = Tokenizer(num_words=MY_VOCAB,
                    oov_token='OOV')
print(A_token.word_index)
A_token.fit_on_texts(articles)
print(A_token.word_index)

#숫자를 단어로 전환 
#sequence: 숫자, text: 단어 
#OOV: out of vocabulary 특수 문자 
print(A_token.sequences_to_texts([[1]]))
print(A_token.sequences_to_texts([[1012]]))

print(A_token.texts_to_sequences(['the']))

# 모든 제외어를 뺀 기사를 tokenizing
A_tokenized= A_token.texts_to_sequences(articles)



28612, 'deodorant': 28613, 'mister': 28614, '449': 28615, 'owing': 28616, 'swelled': 28617, 'babacan': 28618, 'accommodated': 28619, 'substitutions': 28620, 'sniffer': 28621, 'guti': 28622, 'ser': 28623, 'nihat': 28624, 'kahveci': 28625, 'albacete': 28626, 'separatist': 28627, 'accosiation': 28628, 'shamil': 28629, 'fabrication': 28630, 'raps': 28631, 'overcharged': 28632, 'departing': 28633, 'hassall': 28634, 'illston': 28635, 'evidentiary': 28636, 'admissibility': 28637, 'vima': 28638, 'childish': 28639, '짙49m': 28640, 'paion': 28641, 'clotting': 28642, 'cursed': 28643, 'werewolf': 28644, 'ranger': 28645, 'cheerleader': 28646, 'elise': 28647, 'philandering': 28648, 'madea': 28649, 'powerpoint': 28650, 'telemedicine': 28651, 'personalized': 28652, 'hatched': 28653, 'designate': 28654, 'prassana': 28655, 'rambathla': 28656, 'choke': 28657, 'foolproof': 28658, 'starbuck': 28659, 'intermittency': 28660, 'pctvts': 28661, 'insect': 28662, 'postulating': 28663, 'numbered': 28664, 'ecosystem

In [69]:
#tokenizing(Text-2-Sequence)) Output -> 각 word를 정수로 
sample = A_tokenized[MY_SAMPLE]
print(sample)

#기사 통계 내기 
longest = max([len(x) for x in A_tokenized])
print('제일 긴 기사 단어 수:', longest)

shortest = min([len(x) for x in A_tokenized])
print('제일 짧은 기사 단어 수:', shortest)


[1754, 1, 689, 2077, 33, 884, 118, 1041, 76, 1754, 1, 2, 689, 34, 49, 10, 67, 2831, 525, 1273, 3008, 3335, 213, 3791, 1, 33, 1, 336, 1324, 3, 1, 293, 232, 388, 67, 1041, 1, 3062, 2690, 464, 1, 62, 4805, 3, 67, 2, 430, 972, 33, 4183, 338, 2303, 33, 119, 718, 953, 3, 1, 314, 39, 1097, 2440, 140, 223, 1260, 1424, 334, 178, 3791, 76, 1560, 2237, 809, 733, 1, 1, 2882, 1275, 1, 1626, 17, 1602, 265, 19, 2872, 380, 875, 53, 502, 834, 1326, 1, 1, 17, 1]
제일 긴 기사 단어 수: 2279
제일 짧은 기사 단어 수: 50


In [70]:
#기사의 길이 모두 맞추기 

#padding: 200개보다 짧은 경우 처리
#pad_sequence 결과는 numpy로 처리
A_tokenized = pad_sequences(A_tokenized,
                            maxlen=MY_LEN,
                            padding='pre',
                            truncating='pre')


# 기사 통계 내기
longest = max([len(x) for x in A_tokenized])
print('제일 긴 기사 단어 수:', longest)

shortest = min([len(x) for x in A_tokenized])
print('제일 짧은 기사 단어 수:', shortest)

print('hash map에 사용된 총 단어 수:', len(A_token.word_counts))

print('최종 처리된 샘플 기사')
print(A_tokenized[MY_SAMPLE])


#95개의 단어를 가졌기 때문에 앞부분(PRE)에 105개의 Zero padding이 된다

제일 긴 기사 단어 수: 200
제일 짧은 기사 단어 수: 200
hash map에 사용된 총 단어 수: 29698
최종 처리된 샘플 기사
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0 1754    1  689 2077   33  884  118
 1041   76 1754    1    2  689   34   49   10   67 2831  525 1273 3008
 3335  213 3791    1   33    1  336 1324    3    1  293  232  388   67
 1041    1 3062 2690  464    1   62 4805    3   67    2  430  972   33
 4183  338 2303   33  119  718  953    3    1  314   39 1097 2440  140
  223 1260 1424  334  178 3791   76 1560 2237  809  733    1    1 2882

In [71]:
#'라벨' tokenization 처리 
C_token = Tokenizer()
C_token.fit_on_texts(labels)
print(labels)

#단어를 숫자로 전환
#texts_to_sequences -> list output
C_tokenized = C_token.texts_to_sequences(labels)
print(C_tokenized)
print(C_token.word_index)

', 'tech', 'politics', 'business', 'sport', 'politics', 'entertainment', 'tech', 'entertainment', 'sport', 'tech', 'business', 'politics', 'entertainment', 'tech', 'politics', 'sport', 'sport', 'business', 'entertainment', 'business', 'business', 'politics', 'business', 'business', 'politics', 'tech', 'entertainment', 'politics', 'entertainment', 'entertainment', 'business', 'politics', 'sport', 'politics', 'entertainment', 'sport', 'entertainment', 'politics', 'entertainment', 'entertainment', 'entertainment', 'business', 'sport', 'politics', 'sport', 'entertainment', 'tech', 'tech', 'business', 'politics', 'entertainment', 'business', 'sport', 'business', 'politics', 'politics', 'business', 'sport', 'tech', 'tech', 'tech', 'business', 'politics', 'politics', 'business', 'sport', 'sport', 'entertainment', 'business', 'sport', 'sport', 'politics', 'politics', 'politics', 'entertainment', 'politics', 'business', 'tech', 'business', 'politics', 'entertainment', 'entertainment', 'business

In [72]:
#데이터 4분할 
#print(type(A_tokenized))
#print(type(C_tokenized))

#list를 numpy로 전환
C_tokenized = np.array(C_tokenized)
#print(type(C_tokenized))


# 4분할 
X_train, X_test, Y_train, Y_test = train_test_split(A_tokenized,
                                                    C_tokenized,
                                                    train_size=MY_SPLIT,
                                                    shuffle=False)

# 데이터 shape 확인
print('학습용 입력 데이터 shape:', X_train.shape)
print('학습용 출력 데이터 shape:', Y_train.shape)

print('평가용 입력 데이터 shape:', X_test.shape)
print('평가용 출력 데이터 shape:', Y_test.shape)

# 샘플 출력
print("MY_SAMPLE = ",MY_SAMPLE)
print(X_train[MY_SAMPLE])
print(Y_train[MY_SAMPLE])
print(X_test[MY_SAMPLE])
print(Y_test[MY_SAMPLE])


학습용 입력 데이터 shape: (1780, 200)
학습용 출력 데이터 shape: (1780, 1)
평가용 입력 데이터 shape: (445, 200)
평가용 출력 데이터 shape: (445, 1)
MY_SAMPLE =  121
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0 1754    1  689 2077   33  884  118
 1041   76 1754    1    2  689   34   49   10   67 2831  525 1273 3008
 3335  213 3791    1   33    1  336 1324    3    1  293  232  388   67
 1041    1 3062 2690  464    1   62 4805    3   67    2  430  972   33
 4183  338 2303   33  119  718  953    3    1  314   39 1097 2440  140
  223 1260 1424  

In [73]:
#신경망 구현 


In [77]:
#RNN 구현 


model = keras.Sequential()

print('MY_VOCAB:',MY_VOCAB) #5000
print('MY_EMBED:',MY_EMBED) #64
print('MY_HIDDEN:',MY_HIDDEN) #100

model.add(layers.Embedding(input_dim=MY_VOCAB,
                    output_dim=MY_EMBED))

model.add(layers.Dropout(rate=0.5))

#this is the secret sauce!
model.add(layers.LSTM(units=MY_HIDDEN))

model.add(layers.Dense(units=6, activation='softmax'))


print('RNN 요약')
model.summary()


MY_VOCAB: 5000
MY_EMBED: 64
MY_HIDDEN: 100
RNN 요약
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 64)          320000    
_________________________________________________________________
dropout_4 (Dropout)          (None, None, 64)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               66000     
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 606       
Total params: 386,606
Trainable params: 386,606
Non-trainable params: 0
_________________________________________________________________


In [75]:
#인공신경망 학습

In [78]:
#RNN 학습 환경 설정 

model.compile(optimizer=keras.optimizers.Adam(),
                loss= keras.losses.SparseCategoricalCrossentropy(from_logits=True)
                ,metrics=['acc'])


print('MY_EPOCH:',MY_EPOCH)
print('학습 시작')
begin = time()
model.fit(X_train,
          Y_train,
          epochs=MY_EPOCH,
          verbose=1)re
end = time()

print('총 학습 시간: {:.2f}'.format(end-begin))

MY_EPOCH: 10
학습 시작
Epoch 1/10
C:\Users\PC\anaconda3\envs\my_nlp_env\lib\site-packages\tensorflow\python\keras\backend.py:4929: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(


CancelledError:  [_Derived_]RecvAsync is cancelled.
	 [[{{node gradient_tape/sequential_4/embedding_4/embedding_lookup/Reshape/_30}}]] [Op:__inference_train_function_18255]

Function call stack:
train_function
